In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [5]:
df = sns.load_dataset('tips')

In [6]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [8]:
df.duplicated().sum()

1

In [10]:
df.drop_duplicates(inplace=True)

In [14]:
df.shape

(243, 7)

In [16]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
total_bill,243.0,19.813868,8.910071,3.07,13.38,17.81,24.175,50.81
tip,243.0,3.002387,1.385002,1.00,2.00,2.92,3.575,10.00
size,243.0,2.572016,0.952356,1.00,2.00,2.00,3.000,6.00


In [73]:
from sklearn.model_selection import train_test_split   # dividing data into training and for testing 
from sklearn.preprocessing import LabelEncoder   #convert categorical into int
from sklearn.impute import SimpleImputer    #handaling missing values
from sklearn.pipeline import Pipeline   #for automation
from sklearn.preprocessing import StandardScaler       #standardizig
from sklearn.preprocessing import OneHotEncoder #convert categorical to numerical
from sklearn.compose import ColumnTransformer   #differnt preprocessing steps to different columns
from sklearn.ensemble import RandomForestRegressor  #algorithms or Model
from sklearn.linear_model import LinearRegression   #algorithms or Model
from sklearn.tree import DecisionTreeRegressor  #algorithms or Model
from sklearn.metrics import mean_squared_error, r2_score    #To check performance of model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV   #for hyperpyrameter tuning


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 243 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  243 non-null    float64 
 1   tip         243 non-null    float64 
 2   sex         243 non-null    category
 3   smoker      243 non-null    category
 4   day         243 non-null    category
 5   time        243 non-null    category
 6   size        243 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 9.1 KB


In [28]:
categorical_col = ['sex', 'smoker', 'day', 'time']
numerical_col = ['tip', 'size']

In [35]:
# Feature engeniring automation 
# numerical Pipeline 
num_pipeline = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='median')),  #Handaling missing values
    ('scaler', StandardScaler())    #feature scaling
])

# categorical Pipeline 
cat_pipeline = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('oneHotEncoder', OneHotEncoder(handle_unknown='ignore'))
])

In [38]:
# combine into column transfer 
preprocessor = ColumnTransformer([
    ('num', num_pipeline, numerical_col),
    ('catego', cat_pipeline, categorical_col)
])

In [71]:
#Define models and hyperparameter grids
models = {
    'Linear Regression': {
        'model': LinearRegression(),
        'params': {}
        },
    'Random Forest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {
            'regressor__n_estimators': [50,100,150],
            'regressor__max_depth':[None, 5, 10],
            'regressor__min_samples_split': [2, 5]
        }
    },
    'Decision Tree': {
        'model' : DecisionTreeRegressor(random_state=42),
        'params': {
            'regressor__n_estimators': [50,100,150],
            'regressor__max_depth': [None, 5, 10],
            'regressor__min_samples_split': [2,5]
        }
    },
   
    
}

In [43]:
# Features and target 
X = df.drop('total_bill', axis=1)
y = df['total_bill']

In [44]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [76]:
results = {}

for name, config in models.items():
    pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', config['model'])
    ])

    grid = GridSearchCV(pipe, config['params'], cv=5, scoring='r2', n_jobs=-1)
    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)


    results[name] = {
        'Best Params': grid.best_params_,
        'R2 score': r2,
        'MSE':mse
    }

ValueError: Invalid parameter 'n_estimators' for estimator DecisionTreeRegressor(random_state=42). Valid parameters are: ['ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'monotonic_cst', 'random_state', 'splitter'].

In [ ]:
#Print results
print('Model performance comparision:\n')
for name, score in results.items():
    print(f"{name: <20} --> R2 Score: {score['R2 Score']:.4f}, MSE:{score['MSE']:.4f}")
    print(f"Best Params: {score['Best Params', 'N/A']}")

KeyError: 'R2 Score'